* Base Source: https://www.kaggle.com/wangsg/a-self-attentive-model-for-knowledge-tracing
* My First Work: https://www.kaggle.com/leadbest/sakt-self-attentive-knowledge-tracing-submitter

In [34]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [35]:
import gc
import random
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [36]:
#HDKIM
MAX_SEQ = 100
#HDKIMHDKIM

## Load data

In [51]:
%%time
dtype = {'timestamp': 'int64', 
         'user_id': 'int32' ,
         'content_id': 'int16',
         'content_type_id': 'int8',
         'answered_correctly':'int8',
         'prior_question_elapsed_time':'float32'}

train_df = pd.read_csv('/Users/hesu/Documents/kaggle/riiid/riiid-test-answer-prediction/user_seq/train_100_valid_2.csv', usecols=[ 2, 3,4,5,8,9], dtype=dtype)
train_df.head()

CPU times: user 9.4 s, sys: 972 ms, total: 10.4 s
Wall time: 10.5 s


In [52]:
train_df = train_df[train_df.content_type_id == False]

#arrange by timestamp
train_df = train_df.sort_values(['timestamp'], ascending=True).reset_index(drop = True)

In [53]:
train_df.head(10)

,timestamp,user_id,content_id,content_type_id,answered_correctly,prior_question_elapsed_time
0,0,841499248,6054,0,0,NaN
1,0,1123190515,5609,0,1,NaN
2,0,1122899488,3969,0,1,NaN
3,0,1122885826,6314,0,1,NaN
4,0,1874157579,4427,0,1,NaN
5,0,1123820777,3757,0,0,NaN
6,0,1921197508,4102,0,0,NaN
7,0,1123946933,6036,0,1,NaN
8,0,276104471,4181,0,0,NaN
9,0,1123723759,4561,0,1,NaN


In [54]:
elapsed_mean = train_df.prior_question_elapsed_time.mean()

In [55]:
train_df['prior_question_elapsed_time'].fillna(elapsed_mean, inplace=True)


In [56]:
def get_elapsed_time(ela):
    ela = ela // 1000
    if ela > 300:
        return 300
    else:
        return ela

In [57]:
train_df['prior_question_elapsed_time'] = train_df['prior_question_elapsed_time'].apply(lambda x: get_elapsed_time(x))

In [58]:
# question_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')
# question_df['part'] = question_df['part'] - 1
# train_df = pd.merge(train_df, question_df, left_on='content_id',right_on='question_id',how='left')
# train_df['part'].fillna(4, inplace=True)

In [59]:
question_df = pd.read_csv('/Users/hesu/Documents/kaggle/riiid/riiid-test-answer-prediction/questions.csv',usecols=[0,3])
question_df['part'].fillna(4, inplace=True)
question_df['part'] = question_df['part']-1

## Preprocess

In [60]:
skills = train_df["content_id"].unique()
n_skill = 13523
print("number skills", len(skills))

number skills 13516


In [61]:
part_skill = question_df['part'].unique()
n_part_skill = len(part_skill)
print("number part skills", n_part_skill)
print("part is:",part_skill)

number part skills 7
part is: [0 1 2 3 4 5 6]


In [62]:
ela_skill = train_df['prior_question_elapsed_time'].unique()
n_ela_skill = len(ela_skill)
print("number ela skills", n_ela_skill)
print("ela is:", ela_skill)

number ela skills 301
ela is: [ 24.  89.  19.  23.  70.  83.  30.   4.  68.  25.  47.  18.  58.  40.
  33.  69.  48.  44.  32.  36.  22.   7.   6.  34.  21.  17.  31.  16.
  62.  37.  20.  26.  27.  29.  43.   1.  81.   5.  78.  71.  84.  60.
  56.  46.  67.  13.  55.   2.  72.  54.  15.  28.  38.  35. 162.   0.
  45.  52.  61.  39.  11.  10.  12.  14.  64.   3.  57.  91.  41.  50.
  93.  82.  49.  42.   8.  63.   9.  53.  90.  94.  74.  75.  76.  65.
  59.  77.  51. 300. 187. 188. 155. 170. 156. 160.  86. 126. 100. 134.
 217. 218.  73. 183. 216. 204.  80.  98.  87.  96. 154. 148. 141. 241.
 221. 124.  95. 202. 186. 180. 127. 104.  85. 181. 208. 210. 172. 106.
 129. 266. 167. 232.  66. 115. 152. 244.  88. 297. 193. 132. 110. 116.
 196. 215. 219. 139.  79. 228. 179. 117. 105. 113.  99. 112. 102.  92.
 225. 136. 174. 250. 159. 207. 138. 251. 213. 201. 175. 199. 192. 128.
 107. 264. 246. 171. 125. 165. 120. 119. 149.  97. 230. 135. 206. 114.
 109. 111. 191. 158. 103. 209. 123. 131. 198. 2

In [63]:
len(train_df)

21537330

In [64]:
len(train_df)

21537330

In [65]:
train_df = pd.merge(train_df, question_df, left_on='content_id',right_on='question_id',how='left')

In [66]:
group = train_df[['user_id', 'content_id', 'answered_correctly','prior_question_elapsed_time']].groupby('user_id').apply(lambda r: (
            r['content_id'].values,
            r['answered_correctly'].values,
            r['prior_question_elapsed_time'].values))



In [67]:
del train_df
gc.collect()

120

In [68]:
group.head(10)

user_id
115      ([5692, 5716, 128, 7860, 7922, 156, 51, 50, 78...
124      ([7900, 7876, 175, 1278, 2063, 2064, 2065, 336...
2746     ([5273, 758, 5976, 236, 404, 382, 405, 873, 53...
5382     ([5563, 6673, 3859, 9778, 3644, 4640, 6155, 96...
8623     ([5498, 6102, 4078, 4536, 1291, 1131, 1145, 10...
8701     ([3901, 6671, 4963, 6143, 8279, 3964, 4002, 75...
12741    ([7399, 7396, 7397, 7398, 7438, 7439, 7265, 72...
13134    ([11247, 12067, 12005, 12089, 11874, 11956, 12...
24418    ([5851, 5375, 9432, 5060, 8484, 6037, 8674, 43...
24600    ([7900, 7876, 175, 1278, 2065, 2063, 2064, 336...
dtype: object

In [69]:
valid = pd.read_csv('/Users/hesu/Documents/kaggle/riiid/riiid-test-answer-prediction/user_seq/valid_2.csv', usecols=[2,3,4,5,8,9], dtype=dtype)

valid = pd.merge(valid, question_df, left_on='content_id',right_on='question_id',how='left')

In [70]:
class SAKTDataset(Dataset):
    def __init__(self, group, n_skill,n_ela_skill, max_seq=MAX_SEQ): #HDKIM 100
        super(SAKTDataset, self).__init__()
        self.max_seq = max_seq
        self.n_skill = n_skill
        self.n_ela_skill = n_ela_skill
        self.samples = group
        
#         self.user_ids = [x for x in group.index]
        self.user_ids = []
        for user_id in group.index:
            q, qa,ela = group[user_id]
            if len(q) < 5: #HDKIM 10
                continue
            self.user_ids.append(user_id)
            
            #HDKIM Memory reduction
            if len(q)>self.max_seq:
                group[user_id] = (q[-self.max_seq:],qa[-self.max_seq:], ela[-self.max_seq:])

    def __len__(self):
        return len(self.user_ids)

    def __getitem__(self, index):
        user_id = self.user_ids[index]
        q_, qa_,ela_ = self.samples[user_id]
        seq_len = len(q_)

        q = np.zeros(self.max_seq, dtype=int)
        qa = np.zeros(self.max_seq, dtype=int)
        ela = np.zeros(self.max_seq, dtype=int)

        if seq_len >= self.max_seq:
            q[:] = q_[-self.max_seq:]
            qa[:] = qa_[-self.max_seq:]
            ela[:] = ela_[-self.max_seq:]
        else:
            q[-seq_len:] = q_
            qa[-seq_len:] = qa_
            ela[-seq_len:] = ela_   
            
        target_id = q[1:]
        label = qa[1:]
        ela_target = ela[1:]

        x = np.zeros(self.max_seq-1, dtype=int)
        x = q[:-1].copy()
        x += (qa[:-1] == 1) * self.n_skill
        
        ela_x = np.zeros(self.max_seq-1, dtype=int)
        ela_x = ela[:-1].copy()
        ela_x += (qa[:-1]==1) * self.n_ela_skill

        return x, target_id,ela_x,ela_target,label
    # x和target_id都是来自于q，存在一个错位，x比target_id提前一个位置
    # 所以我们可以尝试将ela 也分为ela_x和ela_target_id

In [71]:
dataset = SAKTDataset(group, n_skill, n_ela_skill)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=8)

item = dataset.__getitem__(5)


In [72]:
print(len(item[0]))
print(len(item[1]))
print(len(item[2]))
print(len(item[3]))
print(len(item[4]))

99
99
99
99
99


In [73]:
class ValidDataset(Dataset):
    def __init__(self, samples, test_df, n_skill,n_ela_skill, max_seq=MAX_SEQ): #HDKIM 100
        super(ValidDataset, self).__init__()
        self.samples = samples
        self.user_ids = [x for x in test_df["user_id"].unique()]
        self.test_df = test_df
        self.n_ela_skill = n_ela_skill
        self.n_skill = n_skill
        self.max_seq = max_seq

    def __len__(self):
        return self.test_df.shape[0]

    def __getitem__(self, index):
        test_info = self.test_df.iloc[index]

        user_id = test_info["user_id"]
        target_id = test_info["content_id"]
        ela_target_id = test_info["prior_question_elapsed_time"]
        label = test_info['answered_correctly']

        q = np.zeros(self.max_seq, dtype=int)
        qa = np.zeros(self.max_seq, dtype=int)
        ela = np.zeros(self.max_seq, dtype=int)

        if user_id in self.samples.index:
            q_, qa_, ela_ = self.samples[user_id]
            
            seq_len = len(q_)

            if seq_len >= self.max_seq:
                q = q_[-self.max_seq:]
                qa = qa_[-self.max_seq:]
                ela = ela_[-self.max_seq:]
            else:
                q[-seq_len:] = q_
                qa[-seq_len:] = qa_       
                ela[-seq_len:] = ela_
        
        x = np.zeros(self.max_seq-1, dtype=int)
        x = q[1:].copy()
        x += (qa[1:] == 1) * self.n_skill
        
        questions = np.append(q[2:], [target_id])
        
        ela_x = np.zeros(self.max_seq-1, dtype=int)
        ela_x = ela[1:].copy()
        ela_x += (qa[1:] == 1) * self.n_ela_skill
        
        ela_target = np.append(ela[2:],[ela_target_id])
        
        return x, questions, ela_x, ela_target, np.array([label])

In [74]:
test_dataset = ValidDataset(group, valid, n_skill, n_ela_skill)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False)

## Define model

In [80]:
class FFN(nn.Module):
    def __init__(self, state_size=200):
        super(FFN, self).__init__()
        self.state_size = state_size

        self.lr1 = nn.Linear(state_size, state_size)
        self.relu = nn.ReLU()
        self.lr2 = nn.Linear(state_size, state_size)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        return self.dropout(x)

def future_mask(seq_length):
    future_mask = np.triu(np.ones((seq_length, seq_length)), k=1).astype('bool')
    return torch.from_numpy(future_mask)


class SAKTModel(nn.Module):
    def __init__(self, n_skill,n_ela_skill, max_seq=MAX_SEQ, embed_dim=128): #HDKIM 100
        super(SAKTModel, self).__init__()
        self.n_skill = n_skill
        self.n_ela_skill = n_ela_skill
        self.embed_dim = embed_dim

        self.embedding = nn.Embedding(2*n_skill+1, embed_dim)
        self.pos_embedding = nn.Embedding(max_seq-1, embed_dim)
        self.e_embedding = nn.Embedding(n_skill+1, embed_dim)
        
        self.ela_ans_embedding = nn.Embedding(2*n_ela_skill+1, embed_dim)
        self.ela_embedding = nn.Embedding(n_ela_skill+1, embed_dim)

        self.multi_att = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=8, dropout=0.2)

        self.dropout = nn.Dropout(0.2)
        self.layer_normal = nn.LayerNorm(embed_dim) 

        self.ffn = FFN(embed_dim)
        self.pred = nn.Linear(embed_dim, 1)
    
    def forward(self, x, question_ids, ela_x, ela_target_ids):
        device = x.device        
        print("x.size(1) is:{}".format(x.size(1)))
        x = self.embedding(x)
        pos_id = torch.arange(x.size(1)).unsqueeze(0).to(device)
        
        pos_x = self.pos_embedding(pos_id)
        ela_x = self.ela_ans_embedding(ela_x)
        
        x = x + ela_x + pos_x

        e = self.e_embedding(question_ids)
        ela = self.ela_embedding(ela_target_ids)
        e = e + ela
        
        x = x.permute(1, 0, 2) # x: [bs, s_len, embed] => [s_len, bs, embed]
        e = e.permute(1, 0, 2)
        att_mask = future_mask(x.size(0)).to(device)
        att_output, att_weight = self.multi_att(e, x, x, attn_mask=att_mask)
        # 这个att_output其实是x的一个attention表示
        att_output = self.layer_normal(att_output + e)
        att_output = att_output.permute(1, 0, 2) # att_output: [s_len, bs, embed] => [bs, s_len, embed]

        x = self.ffn(att_output)
        x = self.layer_normal(x + att_output)
        x = self.pred(x)

        return x.squeeze(-1), att_weight

In [81]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SAKTModel(n_skill,n_ela_skill, embed_dim=128)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.99, weight_decay=0.005)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

model.to(device)
criterion.to(device)

BCEWithLogitsLoss()

In [82]:
def train_epoch(model, train_iterator, optim, criterion, device="cpu"):
    model.train()

    train_loss = []
    num_corrects = 0
    num_total = 0
    labels = []
    outs = []

    tbar = tqdm(train_iterator)
    for item in tbar:
        x = item[0].to(device).long()
        target_id = item[1].to(device).long()
        ela_x = item[2].to(device).long()
        ela_target = item[3].to(device).long()
        
        label = item[4].to(device).float()

        optim.zero_grad()
        output, atten_weight = model(x, target_id, ela_x, ela_target)
        loss = criterion(output, label)
        loss.backward()
        optim.step()
        train_loss.append(loss.item())

        output = output[:, -1]
        label = label[:, -1] 
        pred = (torch.sigmoid(output) >= 0.5).long()
        
        num_corrects += (pred == label).sum().item()
        num_total += len(label)

        labels.extend(label.view(-1).data.cpu().numpy())
        outs.extend(output.view(-1).data.cpu().numpy())

        tbar.set_description('loss - {:.4f}'.format(loss))

    acc = num_corrects / num_total
    auc = roc_auc_score(labels, outs)
    loss = np.mean(train_loss)

    return loss, acc, auc

In [83]:
def evaluate_epoch(model, test_dataloader, criterion, device="cpu"):
    model.eval()
    eval_loss = []
    num_corrects = 0
    num_total = 0
    labels = []
    outs = []
    
    with torch.no_grad():
        for item in tqdm(test_dataloader):
            x = item[0].to(device).long()
            target_id = item[1].to(device).long()
            ela_x = item[2].to(device).long()
            ela_target = item[3].to(device).long()
        
            label = item[4].to(device).float()
#             print("In evaluation x shape is:{}\t taret_id shape is:{}\t label shape is:{}".format(x.shape, target_id.shape,label.shape))
            
            output, atten_weight = model(x, target_id, ela_x, ela_target)
            
            output_loss = output[:, -1:]
#             print("Output shape is:{}\tlabel shape is:{}".format(output.shape, label.shape))
            loss = criterion(output_loss, label)
            eval_loss.append(loss.item())
            
            output = output[:,-1]
            label = label[:, -1] 
            pred = (torch.sigmoid(output) >= 0.5).long()
            
            num_corrects += (pred == label).sum().item()
            num_total += len(label)

            labels.extend(label.view(-1).data.cpu().numpy())
            outs.extend(output.view(-1).data.cpu().numpy())

    acc = num_corrects / num_total
#     print("In evaluate labels is:{}\t outs is:{}\t".format(labels, outs))

    auc = roc_auc_score(labels, outs)
#     auc = 0

    loss = np.mean(eval_loss)   
    
    return loss, acc, auc

In [85]:
epochs = 3
best_auc = 0.0
for epoch in range(epochs):
    loss, acc, auc = train_epoch(model, dataloader, optimizer, criterion, device)
    print("epoch - {} train_loss - {:.2f} acc - {:.3f} auc - {:.3f}".format(epoch, loss, acc, auc))
    
    eval_loss, eval_acc, eval_auc = evaluate_epoch(model, test_dataloader, criterion, device)
    print("epoch - {} eval_loss - {:.2f} eval acc - {:.3f} eval auc - {:.3f}".format(epoch, eval_loss, eval_acc, eval_auc))
    if best_auc < eval_auc:
        torch.save(model.state_dict(),'best_model.pt')





  0%|          | 0/196682 [00:00<?, ?it/s]



loss - 0.3862:   0%|          | 0/196682 [00:00<?, ?it/s]

x.size(1) is:99






loss - 0.3862:   0%|          | 1/196682 [00:00<18:11:03,  3.00it/s]



loss - 0.1930:   0%|          | 1/196682 [00:00<18:11:03,  3.00it/s]



loss - 0.3650:   0%|          | 1/196682 [00:00<18:11:03,  3.00it/s]



loss - 0.3650:   0%|          | 3/196682 [00:00<13:52:20,  3.94it/s]



loss - 0.0685:   0%|          | 3/196682 [00:00<13:52:20,  3.94it/s]

x.size(1) is:99
x.size(1) is:99
x.size(1) is:99






loss - 0.2506:   0%|          | 3/196682 [00:00<13:52:20,  3.94it/s]



loss - 0.2506:   0%|          | 5/196682 [00:00<10:48:41,  5.05it/s]



loss - 0.3326:   0%|          | 5/196682 [00:00<10:48:41,  5.05it/s]

...





loss - 0.0837:   1%|          | 1360/196682 [02:03<7:51:24,  6.91it/s]



loss - 0.0837:   1%|          | 1361/196682 [02:03<7:42:10,  7.04it/s]



loss - 0.3356:   1%|          | 1361/196682 [02:03<7:42:10,  7.04it/s]



loss - 0.3356:   1%|          | 1362/196682 [02:03<7:42:07,  7.04it/s]

x.size(1) is:99
x.size(1) is:99






loss - 0.5691:   1%|          | 1362/196682 [02:03<7:42:07,  7.04it/s]



loss - 0.5691:   1%|          | 1363/196682 [02:03<7:59:18,  6.79it/s]

x.size(1) is:99


KeyboardInterrupt: 

In [35]:
import gc
del dataset
gc.collect()

20

## Test

In [38]:
model = SAKTModel(n_skill, embed_dim=128)
model.load_state_dict(torch.load('best_model.pt',map_location=device))
model.to(device)

TypeError: __init__() missing 1 required positional argument: 'n_ela_skill'